# NB. this is a Raku notebook!

The purpose of this notebook is to illustrate a raku-centric hybrid workflow:

1. Use raku Dan for data acquisition & cleansing
2. Use raku Dan::Pandas create a shadow Python Pandas dataframe
3. Pass Dan::Pandas reference to python Matplotlib & Seaborn to create plots (via Inline::Python)

See other notebooks for raku to python hybrid workflow.

In [10]:
use Dan;
use Dan::Pandas;

use Text::CSV;
my @lines  = csv( in => 'DemographicData.csv' );

my \stats = Dan::DataFrame.new( columns => @lines[0], data => @lines[1..*-1] );

stats.columns = <CountryName CountryCode BirthRate InternetUsers IncomeGroup> Z=> 0..∞;
stats.data[*;2,3].map({$_.=Rat});

~stats.describe;

        BirthRate           InternetUsers      
 count  195                 195                
 mean   21.469928           42.076470891949    
 std    1.0025740068320432  1.0025740068320432 
 min    7.9                 0.9                
 25%    12.1205             14.52              
 50%    19.68               41                 
 75%    29.7595             66.225             
 max    49.661              96.5468            

In [26]:
my $stats-shadow = Dan::Pandas::DataFrame.new( stats );

say $stats-shadow.^name;
say $stats-shadow.po;

Dan::Pandas::DataFrame
Inline::Python::PythonObject.new(ptr => NativeCall::Types::Pointer.new(140108668532624), python => Inline::Python.new)


In [28]:
use Inline::Python;

my $mpl-start-str = qq{
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 8,4
import warnings
warnings.filterwarnings('ignore')
};

my $py-sesh = Inline::Python.new;

signal only works in main thread of the main interpreter